```python
mlflow.evaluate(model=None, data=None, *, model_type=None, targets=None, predictions=None, dataset_path=None, feature_names=None, evaluators=None, evaluator_config=None, custom_metrics=None, extra_metrics=None, custom_artifacts=None, validation_thresholds=None, baseline_model=None, env_manager='local', model_config=None, baseline_config=None, inference_params=None)
```


In [ ]:
import mlflow
mlflow.login()

In [ ]:
experiment_name = "/Shared/Experiments/White Wine Quality"
try:
    mlflow.create_experiment(
        name = experiment_name,
        tags={
            "mlflow.note.content": "This experiment is for the white wine quality dataset. The goal is to predict the quality of the wine based on the features provided in the dataset.",
            "project_name": "White Wine Quality Prediction"
        }
    )
    experiment = mlflow.get_experiment_by_name(experiment_name)
except mlflow.exceptions.MlflowException:
    print("Experiment already exists")
    experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)


In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from mlflow.data.pandas_dataset import PandasDataset
from sklearn.datasets import load_iris
data = load_iris(as_frame=True)

# Extract the features and target data separately
y = data.target
X = data.data

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=17
)

# Create a label encoder object
le = LabelEncoder()

# Fit an XGBoost binary classifier on the training data split
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Build the Evaluation Dataset from the test set
y_test_pred = model.predict(X=X_test)

eval_data = X_test
eval_data["target"] = y_test


# Create the PandasDataset for use in mlflow evaluate
pd_dataset = mlflow.data.from_pandas(
    eval_data, predictions=None, targets="target"
)

# Log the Dataset, model, and execute an evaluation run using the configured Dataset
with mlflow.start_run() as run:
    mlflow.log_input(pd_dataset, context="training")

    mlflow.sklearn.log_model(
        artifact_path="white-wine-rfc", sk_model=model, input_example=X_test
    )

    model_uri = f"runs:/{run.info.run_id}/white-wine-rfc"
    result = mlflow.evaluate(model=model_uri, data=pd_dataset, predictions=None, model_type="classifier")
